In [1]:
import pandas
import numpy
import scipy
import math
import scipy.stats
import seaborn

In [2]:
# module for figures
import matplotlib, matplotlib.pyplot as plt

# this is a trick to make figures look nicer
matplotlib.rcParams.update({'font.size':20, 'font.family':'FreeSans', 'xtick.labelsize':20, 'ytick.labelsize':10, 'figure.figsize':(12, 8)})

# 0. user-defined variables

In [3]:
input_file_directory = '/home/adrian/projects/ATG7/data/'

normal_tissue_expression_file = '/home/adrian/projects/ATG7/results/normal_tissue_expression.tsv'
primary_tumor_expression_file = '/home/adrian/projects/ATG7/results/primary_tumors_expression.tsv'

## functions & options


In [4]:
pandas.set_option("max_rows", None)
# pandas.reset_option("max_rows")

In [5]:
#define a function to split after the ".". The 1 is for how many word to have. if we have several dot. The [1] is what is after the dot, and [0] what is before
def split(L):
  return L.split(".",1)[0]

In [6]:
def binarise_mutation(DF):
  old = ['no variant', "3'UTR", "5'UTR", 'Intron', "5'Flank", 'Silent', 
         'large deletion', 'Nonstop_Mutation','Missense_Mutation', 'Nonsense_Mutation', 'Frame_Shift_Del', 'Frame_Shift_Ins', 'Splice_Site', 'In_Frame_Del', 'In_Frame_Ins', 'Translation_Start_Site']
  new = [0,0,0,0,0,0,
         1,1,1,1,1,1,1,1,1,1]

  for i in range(len(old)):
    DF['effect'] = DF['effect'].replace(old[i], new[i])


# II] Dataframe settings

## A) Isoform data

In [7]:
# Original data isoforms
directory = input_file_directory + "xena_surv_ATG7.tsv"
df_ori= pandas.read_csv(directory, sep = "\t")


In [8]:
# Column selection
df= df_ori[['sample',
            '_sample_type',
            '_primary_site',
            'ENSG00000197548.12',
            'ENST00000354449.7', 
            'ENST00000354956.9',
            'ENST00000446450.6']]

# We have Adrenal Gland and Adrenal gland. I need to put "G".
df['_primary_site'] = df['_primary_site'].str.title()

# Column rename
df.rename(columns = {'sample' : 'sample',
    '_sample_type' : 'Sample_Type', 
                            '_primary_site' : 'Primary_Site', 
                            'ENST00000354449.7': 'ATG7_1',
                            'ENST00000354956.9': 'ATG7_2',
                            'ENST00000446450.6': 'ATG7_3',
                            'ENSG00000197548.12': 'ATG7'}, inplace = True)

# Calcul to have value from log2;  2**(ATG7(1))-0.001
df["ATG7(total)"] = pow(2,df['ATG7'])-0.001
df["ATG7(1)"] = pow(2,df['ATG7_1'])-0.001
df["ATG7(2)"] = pow(2,df['ATG7_2'])-0.001
df["ATG7(3)"] = pow(2,df['ATG7_3'])-0.001

# Calcul to have log2+1 from value; 
df["log2_+1_ATG7(total)"] = numpy.log2(df['ATG7(total)']+1)
df["log2_+1_ATG7(1)"] = numpy.log2(df['ATG7(1)']+1)
df["log2_+1_ATG7(2)"] = numpy.log2(df['ATG7(2)']+1)

# Calcul to see percentage of expression of the 3 isoforms of ATG7
df['ATG7_prot_tot'] = (df['ATG7(1)'] + df['ATG7(2)'] + df['ATG7(3)'])
df['ATG7_1%'] = (df['ATG7(1)'] *100)/ df['ATG7_prot_tot']
df['ATG7_2%'] = (df['ATG7(2)'] *100)/ df['ATG7_prot_tot']
df['ATG7_3%'] = (df['ATG7(3)'] *100)/ df['ATG7_prot_tot']

#Add columns, normalized on total
df["ATG7(1)norm"] = df['ATG7(1)']/df["ATG7(total)"]
df["ATG7(2)norm"] = df['ATG7(2)']/df["ATG7(total)"]
df.head()

/home/adrian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/adrian/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/adrian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

,sample,Sample_Type,Primary_Site,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7(total),ATG7(1),ATG7(2),ATG7(3),log2_+1_ATG7(total),log2_+1_ATG7(1),log2_+1_ATG7(2),ATG7_prot_tot,ATG7_1%,ATG7_2%,ATG7_3%,ATG7(1)norm,ATG7(2)norm
0,TARGET-20-PARUBT-40,Recurrent Blood Derived Cancer - Peripheral Blood,White Blood Cell,4.401,1.3900,1.522,-1.7320,21.125765,2.619787,2.870889,3.000343e-01,4.467655,1.855905,1.952665,5.790710,45.241201,49.577495,5.181305,0.124009,0.135895
1,TARGET-20-PATJHJ-40,Recurrent Blood Derived Cancer - Peripheral Blood,White Blood Cell,2.820,1.1180,-1.086,-9.9660,7.060624,2.169459,0.470066,-1.495113e-07,3.010892,1.664236,0.555881,2.639524,82.191280,17.808726,-0.000006,0.307262,0.066576
2,TARGET-10-PASLZM-40,Recurrent Blood Derived Cancer - Peripheral Blood,White Blood Cell,1.158,0.0158,-3.171,-3.8160,2.230479,1.010012,0.110028,7.000183e-02,1.691748,1.007204,0.150597,1.190042,84.871949,9.245753,5.882299,0.452823,0.049329
3,TARGET-21-PATAIJ-42,Post treatment Blood Cancer - Blood,White Blood Cell,4.332,1.3450,1.975,-9.9660,20.139115,2.539302,3.930282,-1.495113e-07,4.401843,1.823465,2.301670,6.469584,39.249848,60.750154,-0.000002,0.126088,0.195157
4,TARGET-21-PASVJS-41,Post treatment Blood Cancer - Bone Marrow,White Blood Cell,5.156,1.8040,3.769,0.5069,35.653197,3.490870,13.631706,1.419994e+00,5.195867,2.166995,3.871026,18.542569,18.826249,73.515731,7.658019,0.097912,0.382342


In [9]:
df['Sample_Type'].value_counts()

Primary Tumor                                        9185
Normal Tissue                                        7429
Solid Tissue Normal                                   738
Cell Line                                             433
Metastatic                                            393
Primary Solid Tumor                                   286
Primary Blood Derived Cancer - Peripheral Blood       239
Primary Blood Derived Cancer - Bone Marrow            237
Recurrent Blood Derived Cancer - Bone Marrow          104
Recurrent Tumor                                        45
Recurrent Solid Tumor                                  13
Post treatment Blood Cancer - Bone Marrow              12
Additional - New Primary                               11
Recurrent Blood Derived Cancer - Peripheral Blood       3
Post treatment Blood Cancer - Blood                     1
Control Analyte                                         1
Additional Metastatic                                   1
Name: Sample_T

In [10]:
#Creation of two tables, Normal tissue and Primary tumor.
df_ATG7_Normal = df[df['Sample_Type'].str.match ('Normal Tissue')]
df_ATG7_Primary = df[df['Sample_Type'].str.match ('Primary Tumor')]

print('normal_tissue' , len(df_ATG7_Normal))
print('primary_tumors' , len(df_ATG7_Primary))

normal_tissue 7429
primary_tumors 9185


In [11]:
# df with only Normal and Primary 
df_Norm_Prim = pandas.concat([df_ATG7_Normal, df_ATG7_Primary], axis=0)
df_Norm_Prim.reset_index(drop=True, inplace=True)
df_Norm_Prim.head(3)

,sample,Sample_Type,Primary_Site,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7(total),ATG7(1),ATG7(2),ATG7(3),log2_+1_ATG7(total),log2_+1_ATG7(1),log2_+1_ATG7(2),ATG7_prot_tot,ATG7_1%,ATG7_2%,ATG7_3%,ATG7(1)norm,ATG7(2)norm
0,GTEX-ZTTD-0326-SM-57WFW,Normal Tissue,Muscle,5.042,3.860,2.390,0.2642,32.944283,14.519306,5.240574,1.199970e+00,5.085097,3.955992,2.641679,20.959850,69.271996,25.002916,5.725088,0.440723,0.159074
1,GTEX-PX3G-1626-SM-2S1PT,Normal Tissue,Muscle,4.252,2.322,1.956,1.1320,19.052710,4.999249,3.878848,2.190624e+00,4.325725,2.584782,2.286540,11.068720,45.165557,35.043325,19.791118,0.262390,0.203585
2,GTEX-OXRO-1726-SM-3LK6C,Normal Tissue,Muscle,3.909,2.602,1.345,-9.9660,15.020948,6.070277,2.539302,-1.495113e-07,4.001888,2.821767,1.823465,8.609579,70.506086,29.493916,-0.000002,0.404121,0.169051


### High/Low expression per Median for Isoform expression

In [12]:
# What is the median for iso 1 and iso 2?
#High >50% and low <50%

In [13]:
Data = [[df_ATG7_Normal, 'Normal'], [df_ATG7_Primary, 'Primary']]
isoforms = [["ATG7_1", "iso1"], ["ATG7_2", "iso2"]]

df_LoHi = [] 

for df in Data:
  for isoform in isoforms:
    xMed = df[0][isoform[0]].median()

    low = df[0].loc[df[0][isoform[0]] < xMed]
    low['L/H'] = "Low_"+isoform[1]

    high = df[0].loc[df[0][isoform[0]] > xMed]
    high['L/H'] = "High_"+isoform[1]

  #merge low and high
  concat = pandas.concat([low, high], axis=0)
  col = concat.pop('L/H')
  concat.insert(3, 'L/H', col)
  df_LoHi.append(concat)

normal_LoHi = df_LoHi[0]
primary_LoHi = df_LoHi[1]

/home/adrian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/adrian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## B) Whole gene expression

In [14]:
#Import list of protein coding transcripts
protein_coding_directory = input_file_directory + "protein_coding_list.txt"
protein_coding_list= pandas.read_csv(protein_coding_directory, sep = "\t")
print(len(protein_coding_list))
protein_coding_list.head(2)

22796


,ensembl_gene_id,hgnc_symbol,entrezgene_id,transcript_biotype
1,ENSG00000198888,MT-ND1,4535.0,protein_coding
2,ENSG00000198763,MT-ND2,4536.0,protein_coding


In [15]:
%%time

df_gexpr_normal= pandas.read_csv(normal_tissue_expression_file, sep = "\t")
df_gexpr_primary= pandas.read_csv(primary_tumor_expression_file, sep = "\t")

CPU times: user 2min 27s, sys: 3.4 s, total: 2min 31s
Wall time: 2min 31s


In [16]:
df_gexpr_normal.head()

,sample,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2226-SM-5N9CH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2826-SM-5GZXL,...,GTEX-ZZPU-0726-SM-5N9C8,GTEX-ZZPU-0926-SM-5GZYT,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
0,ENSG00000242268.2,-9.9658,-9.9658,-9.9658,-1.2481,-3.8160,-1.7809,-9.9658,-9.9658,-3.6259,...,-0.8084,-9.9658,-9.9658,-9.9658,-4.0350,-9.9658,-9.9658,-9.9658,-9.9658,-3.6259
1,ENSG00000259041.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658
2,ENSG00000270112.3,-4.2934,0.0014,-9.9658,-5.5735,0.3573,-9.9658,-6.5064,-5.0116,-9.9658,...,-4.0350,-1.8314,-1.7809,-2.0529,-6.5064,-3.6259,-9.9658,-9.9658,-2.8262,-9.9658
3,ENSG00000167578.16,5.1190,4.1277,4.4067,5.6860,4.0357,4.6849,4.5009,5.3954,4.9402,...,3.9099,3.3478,5.5201,5.6930,4.2072,4.2788,4.0790,4.7856,3.5387,4.7071
4,ENSG00000278814.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658


In [17]:
df_gexpr_primary.head()

,sample,TCGA-02-0047-01,TCGA-02-0055-01,TCGA-02-2483-01,TCGA-02-2485-01,TCGA-04-1331-01,TCGA-04-1332-01,TCGA-04-1337-01,TCGA-04-1338-01,TCGA-04-1341-01,...,TCGA-ZP-A9D4-01,TCGA-ZQ-A9CR-01,TCGA-ZR-A9CJ-01,TCGA-ZS-A9CD-01,TCGA-ZS-A9CE-01,TCGA-ZS-A9CF-01,TCGA-ZS-A9CG-01,TCGA-ZT-A8OM-01,TCGA-ZU-A8S4-01,TCGA-ZX-AA5X-01
0,ENSG00000242268.2,0.0158,-9.9658,-9.9658,-0.6193,-5.0116,-3.4580,-3.8160,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-4.6082,-9.9658,-9.9658,-9.9658,-3.6259,-9.9658,-9.9658
1,ENSG00000259041.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658
2,ENSG00000270112.3,-0.8863,-3.8160,-5.5735,-1.5522,0.4761,-2.4659,-5.0116,-4.0350,-2.7274,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-6.5064
3,ENSG00000167578.16,5.9750,5.2506,5.4519,6.5390,4.9514,6.4481,5.1595,4.0506,4.5249,...,3.6136,4.4700,4.1780,4.5547,3.6737,4.9331,3.7646,5.5201,5.4216,4.7991
4,ENSG00000278814.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658


In [18]:
df_gexpr_normal.head()

,sample,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2226-SM-5N9CH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2826-SM-5GZXL,...,GTEX-ZZPU-0726-SM-5N9C8,GTEX-ZZPU-0926-SM-5GZYT,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
0,ENSG00000242268.2,-9.9658,-9.9658,-9.9658,-1.2481,-3.8160,-1.7809,-9.9658,-9.9658,-3.6259,...,-0.8084,-9.9658,-9.9658,-9.9658,-4.0350,-9.9658,-9.9658,-9.9658,-9.9658,-3.6259
1,ENSG00000259041.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658
2,ENSG00000270112.3,-4.2934,0.0014,-9.9658,-5.5735,0.3573,-9.9658,-6.5064,-5.0116,-9.9658,...,-4.0350,-1.8314,-1.7809,-2.0529,-6.5064,-3.6259,-9.9658,-9.9658,-2.8262,-9.9658
3,ENSG00000167578.16,5.1190,4.1277,4.4067,5.6860,4.0357,4.6849,4.5009,5.3954,4.9402,...,3.9099,3.3478,5.5201,5.6930,4.2072,4.2788,4.0790,4.7856,3.5387,4.7071
4,ENSG00000278814.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658


In [19]:
#Preparation protein coding gene expression
#remove dot in ENSEMBL name. Function split created at the beginning
data = [df_gexpr_normal, df_gexpr_primary]

df_protcoding_exp = []
for df in data: 
  #remove dot in ens name
  df["sample"] = df["sample"].apply(split)
  #see if have duplicate
  print(df.duplicated(subset=['sample']).sum())
    
  print('before')
  print(df.loc[df['sample'] == 'ENSG00000198712'])
  print('')

  #Merge with prot coding list
  df.rename(columns = {'sample' : 'ensembl_gene_id'}, inplace = True)
  dfmerged = protein_coding_list.merge(df)
  df_protcoding_exp.append(dfmerged)

  #control merge is correct
  print("control merge is correct")
  print(df.loc[df['ensembl_gene_id'] == 'ENSG00000198712'])
  #print(df_protcoding_exp[0].loc[df_protcoding_exp[0]['ensembl_gene_id'] == 'ENSG00000198712'])

normal_protcoding = df_protcoding_exp[0]
primary_protcoding = df_protcoding_exp[1]

0
before
                sample  GTEX-1117F-0226-SM-5GZZ7  GTEX-1117F-0426-SM-5EGHI  \
28551  ENSG00000198712                   12.8384                   14.7048   

       GTEX-1117F-0526-SM-5EGHJ  GTEX-1117F-0626-SM-5N9CS  \
28551                   12.8456                   12.6928   

       GTEX-1117F-0726-SM-5GIEN  GTEX-1117F-1326-SM-5EGHH  \
28551                   14.8495                   13.4601   

       GTEX-1117F-2226-SM-5N9CH  GTEX-1117F-2426-SM-5EGGH  \
28551                   11.1638                   12.1732   

       GTEX-1117F-2826-SM-5GZXL  ...  GTEX-ZZPU-0726-SM-5N9C8  \
28551                   13.7657  ...                  13.3606   

       GTEX-ZZPU-0926-SM-5GZYT  GTEX-ZZPU-1226-SM-5N9CK  \
28551                  15.3773                  14.9873   

       GTEX-ZZPU-1326-SM-5GZWS  GTEX-ZZPU-1426-SM-5GZZ6  \
28551                  14.6528                   14.776   

       GTEX-ZZPU-1826-SM-5E43L  GTEX-ZZPU-2126-SM-5EGIU  \
28551                   14.834       

In [20]:
normal_protcoding = df_protcoding_exp[0]
primary_protcoding = df_protcoding_exp[1]

In [21]:
#df_protcoding_exp[df_protcoding_exp['ensembl_gene_id'] == 'ENSG00000198712']

TypeError: list indices must be integers or slices, not str

In [22]:
type(df_protcoding_exp)

list

In [25]:
df_ATG7_Normal.head()

,sample,Sample_Type,Primary_Site,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7(total),ATG7(1),ATG7(2),ATG7(3),log2_+1_ATG7(total),log2_+1_ATG7(1),log2_+1_ATG7(2),ATG7_prot_tot,ATG7_1%,ATG7_2%,ATG7_3%,ATG7(1)norm,ATG7(2)norm
656,GTEX-ZTTD-0326-SM-57WFW,Normal Tissue,Muscle,5.042,3.860,2.390,0.2642,32.944283,14.519306,5.240574,1.199970e+00,5.085097,3.955992,2.641679,20.959850,69.271996,25.002916,5.725088,0.440723,0.159074
657,GTEX-PX3G-1626-SM-2S1PT,Normal Tissue,Muscle,4.252,2.322,1.956,1.1320,19.052710,4.999249,3.878848,2.190624e+00,4.325725,2.584782,2.286540,11.068720,45.165557,35.043325,19.791118,0.262390,0.203585
658,GTEX-OXRO-1726-SM-3LK6C,Normal Tissue,Muscle,3.909,2.602,1.345,-9.9660,15.020948,6.070277,2.539302,-1.495113e-07,4.001888,2.821767,1.823465,8.609579,70.506086,29.493916,-0.000002,0.404121,0.169051
659,GTEX-13JVG-0126-SM-5L3D3,Normal Tissue,Muscle,3.854,2.568,1.428,-9.9660,14.459044,5.928868,2.689734,-1.495113e-07,3.950379,2.792620,1.883517,8.618602,68.791526,31.208476,-0.000002,0.410046,0.186024
660,GTEX-NPJ8-1626-SM-2HMIY,Normal Tissue,Muscle,3.822,2.164,1.293,-1.5100,14.141840,4.480557,2.449371,3.501112e-01,3.920469,2.454322,1.786333,7.280039,61.545783,33.645022,4.809194,0.316830,0.173200


In [26]:
normal_protcoding.head()

,ensembl_gene_id,hgnc_symbol,entrezgene_id,transcript_biotype,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,...,GTEX-ZZPU-0726-SM-5N9C8,GTEX-ZZPU-0926-SM-5GZYT,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
0,ENSG00000198888,MT-ND1,4535.0,protein_coding,13.1350,14.6712,12.8924,12.6263,13.8339,13.7962,...,12.8565,14.8073,14.3240,14.0776,14.1602,13.7105,12.8758,12.5436,14.6088,14.3228
1,ENSG00000198763,MT-ND2,4536.0,protein_coding,12.6215,14.2931,12.6612,13.0833,14.1782,13.8157,...,13.0353,14.6549,14.3492,14.2711,14.3810,14.2432,13.3940,12.8245,14.2491,14.3830
2,ENSG00000198804,MT-CO1,4512.0,protein_coding,12.6246,14.5995,12.5008,11.3334,15.3590,12.2692,...,13.3965,15.9901,15.2750,15.1726,14.8551,14.6084,13.5241,14.0931,15.5290,14.3970
3,ENSG00000198712,MT-CO2,4513.0,protein_coding,12.8384,14.7048,12.8456,12.6928,14.8495,13.4601,...,13.3606,15.3773,14.9873,14.6528,14.7760,14.8340,13.4387,14.1687,15.1614,14.3728
4,ENSG00000228253,MT-ATP8,4509.0,protein_coding,13.9763,15.7851,13.9252,14.1037,16.3241,15.3932,...,14.3118,16.4238,15.5251,15.8844,15.8104,15.4948,14.9172,14.8163,16.0166,15.2887


In [33]:
curr_df

,sample,GTEX-ZTTD-0326-SM-57WFW,GTEX-PX3G-1626-SM-2S1PT,GTEX-OXRO-1726-SM-3LK6C,GTEX-13JVG-0126-SM-5L3D3,GTEX-NPJ8-1626-SM-2HMIY,GTEX-ZC5H-0326-SM-5HL9F,GTEX-ZYY3-0526-SM-5E45G,GTEX-11TTK-0326-SM-5BC5N,GTEX-XV7Q-2926-SM-4BRUL,...,GTEX-ZAJG-2826-SM-5HL8D,GTEX-1399U-1326-SM-5IJET,GTEX-132AR-1526-SM-5KM1L,GTEX-139D8-2526-SM-5N9G3,GTEX-ZV6S-1526-SM-5NQ7O,GTEX-WWYW-2826-SM-4MVOQ,GTEX-YFC4-0126-101855-SM-5CVLZ,GTEX-YEC4-1026-101814-SM-5P9FX,GTEX-YEC3-2526-101809-SM-5CVLT,GTEX-YEC3-1426-101806-SM-5PNXX
4,ATG7_1,3.86,2.322,2.602,2.568,2.164,2.039,1.619,2.269,2.511,...,2.759,2.964,2.628,2.614,2.791,2.566,2.888,2.275,1.683,2.359
5,ATG7_2,2.39,1.956,1.345,1.428,1.293,1.008,1.532,1.967,1.215,...,-9.966,-0.9132,-9.966,-9.966,-3.047,-9.966,1.812,1.745,1.872,-0.3752
6,ATG7_3,0.2642,1.132,-9.966,-9.966,-1.51,-9.966,-0.3752,-1.685,-9.966,...,-9.966,-3.458,-9.966,-9.966,-9.966,-9.966,0.2029,-0.1993,-0.4325,-9.966
7,ATG7(total),32.944283,19.05271,15.020948,14.459044,14.14184,12.517589,12.091159,12.040974,11.711686,...,10.259293,10.202555,9.868657,9.278197,9.018203,8.729111,22.176118,13.094507,12.200613,8.710976
8,ATG7(1),14.519306,4.999249,6.070277,5.928868,4.480557,4.108606,3.070621,4.818889,5.69915,...,6.768269,7.801844,6.180684,6.120987,6.920094,5.920653,7.401435,4.838976,3.20995,5.129146
9,ATG7(2),5.240574,3.878848,2.539302,2.689734,2.449371,2.010121,2.890865,3.908543,2.320408,...,-0.0,0.530006,-0.0,-0.0,0.119993,-0.0,3.510287,3.350949,3.659397,0.769999
10,ATG7(3),1.19997,2.190624,-0.0,-0.0,0.350111,-0.0,0.769999,0.310003,-0.0,...,-0.0,0.089999,-0.0,-0.0,-0.0,-0.0,1.15001,0.869973,0.739977,-0.0
11,log2_+1_ATG7(total),5.085097,4.325725,4.001888,3.950379,3.920469,3.756766,3.710521,3.70498,3.668083,...,3.493044,3.485756,3.442102,3.361515,3.324552,3.282308,4.534567,3.817061,3.722533,3.279616
12,log2_+1_ATG7(1),3.955992,2.584782,2.821767,2.79262,2.454322,2.35293,2.025249,2.540744,2.743978,...,2.957593,3.137806,2.844121,2.832077,2.985517,2.790908,3.070636,2.545715,2.073803,2.615686
13,log2_+1_ATG7(2),2.641679,2.28654,1.823465,1.883517,1.786333,1.589822,1.960091,2.295295,1.73136,...,-0.0,0.613537,-0.0,-0.0,0.16349,-0.0,2.173219,2.12133,2.220143,0.823748


In [34]:
normal_protcoding.head()

,ensembl_gene_id,hgnc_symbol,entrezgene_id,transcript_biotype,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,...,GTEX-ZZPU-0726-SM-5N9C8,GTEX-ZZPU-0926-SM-5GZYT,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
0,ENSG00000198888,MT-ND1,4535.0,protein_coding,13.1350,14.6712,12.8924,12.6263,13.8339,13.7962,...,12.8565,14.8073,14.3240,14.0776,14.1602,13.7105,12.8758,12.5436,14.6088,14.3228
1,ENSG00000198763,MT-ND2,4536.0,protein_coding,12.6215,14.2931,12.6612,13.0833,14.1782,13.8157,...,13.0353,14.6549,14.3492,14.2711,14.3810,14.2432,13.3940,12.8245,14.2491,14.3830
2,ENSG00000198804,MT-CO1,4512.0,protein_coding,12.6246,14.5995,12.5008,11.3334,15.3590,12.2692,...,13.3965,15.9901,15.2750,15.1726,14.8551,14.6084,13.5241,14.0931,15.5290,14.3970
3,ENSG00000198712,MT-CO2,4513.0,protein_coding,12.8384,14.7048,12.8456,12.6928,14.8495,13.4601,...,13.3606,15.3773,14.9873,14.6528,14.7760,14.8340,13.4387,14.1687,15.1614,14.3728
4,ENSG00000228253,MT-ATP8,4509.0,protein_coding,13.9763,15.7851,13.9252,14.1037,16.3241,15.3932,...,14.3118,16.4238,15.5251,15.8844,15.8104,15.4948,14.9172,14.8163,16.0166,15.2887


In [32]:
#Preparation of the global dataframe
# data = [[df_protcoding[0], 'normal_protcoding'], [df_protcoding[1], 'primary_protcoding']]


data = [[df_ATG7_Normal, normal_protcoding], [df_ATG7_Primary, primary_protcoding]]

df_final = []

for df in data:
    
  print(type(df[0]))
  print(df[0].columns)
  print('-----------')  

  #print(df[0].columns()[:10])  
  
  #transpose the table 
  curr_df = df[0].copy()
  curr_df = numpy.transpose(curr_df)

  #Change head column & delete the name of the column index; choose ENSEMBL
  curr_df.columns = curr_df.iloc[0]
  curr_df.columns.name = None
  curr_df = curr_df.reset_index()

  #Delete the first rows & columns rename to merge
  curr_df = curr_df.drop(curr_df.index[0:4])
  curr_df.rename(columns = {'index' : 'sample'}, inplace = True)

  #Merge the two df to have isoform expression + whole gene expression 
  df_merged = pandas.merge(df[1], curr_df, on= 'sample')

  print('df_isoform is: ', len(df_Norm_Prim))
  print('df_gene_exp is: ', len(df))
  print('df_merged: ', len(df_merged))

  #delete the sample column and select column
  df_merged = df_merged.drop("sample", 1)
  df_merged = gexpr_compl.drop(gexpr_compl.columns[5:19], axis=1)
  print("controler si j'ai bien ATG7_2", 'suivi de ENSG00000198888')
  
  #save in list
  df_final.append(df_merged)


df_final[0] = df_normal_expr
df_final[1] = df_primary_expr

<class 'pandas.core.frame.DataFrame'>
Index(['sample', 'Sample_Type', 'Primary_Site', 'ATG7', 'ATG7_1', 'ATG7_2',
       'ATG7_3', 'ATG7(total)', 'ATG7(1)', 'ATG7(2)', 'ATG7(3)',
       'log2_+1_ATG7(total)', 'log2_+1_ATG7(1)', 'log2_+1_ATG7(2)',
       'ATG7_prot_tot', 'ATG7_1%', 'ATG7_2%', 'ATG7_3%', 'ATG7(1)norm',
       'ATG7(2)norm'],
      dtype='object')
-----------


KeyError: 'sample'

# III] ATG7 expression

##A) Distribution


In [ ]:
# Distribution graph, for ATG7, iso1 & iso2

all_dfs = [[df_ATG7_Normal, 'in all tissue', df_ATG7_Primary, 'all primary tumors']]

isoforms = [['log2_+1_ATG7(total)', 'ATG7'], ['log2_+1_ATG7(1)', 'ATG7(1)'], ['log2_+1_ATG7(2)', 'ATG7(2)']]


for df in all_dfs:
  for isoform in isoforms:
    xData = df[0][isoform[0]]
    x2Data = df[2][isoform[0]]

    seaborn.kdeplot(xData, color = 'mediumaquamarine', fill = True)
    seaborn.kdeplot(x2Data, color = 'sandybrown', fill = True)

    plt.xlabel('log2 '+ isoform[1]+ ' +1')
    plt.title(isoform[1]+ ' expression ' + df[1] )
    plt.legend(labels=["Normal Tissue","Primary Tumor "])
    plt.show()
    plt.clf()

##B) Percentage do not express ATG7 isoforms

In [ ]:
#Percentage of patients: 

all_dfs = [[df_ATG7_Normal, 'all normal tissue'], [df_ATG7_Primary, 'all primary tumor']]

isoforms = [['ATG7(total)', 'ATG7'], ['ATG7(1)', 'ATG7(1)'], ['ATG7(2)', 'ATG7(2)'], ['ATG7(3)', 'ATG7(3)']]

for df in all_dfs:
  print(df[1])
  for isoform in isoforms:
        
    QUOI = isoform[1]
    OU = df[1]
    all = (df[0][isoform[0]]).shape
    no_exp = (df[0][isoform[0]]<0.1).sum()
    perc_no_exp = (no_exp*100)/all
    perc_no_exp = float(perc_no_exp)

    # print("The percentage of patient who do not express", QUOI, OU, "is:", round(perc_no_exp,1),'%')
    print(round(perc_no_exp,1),'% of tumor do not express', QUOI)
  print()


## C) Expression figures

### 1) Value TPM

In [ ]:
# Distribution: Data preparation + graph

data = [[df_Norm_Prim, 'all tissue']]

for df in data:
   
  shape = df[0].shape
  shape = shape[0]
  print(shape)

  #select the sample type column
  sample_type = {'sample_type':[df[0].iloc[:,1][i]
      for i in range(0, shape)]}

  #convert to df, and copy it 3 times
  sample_typedf = pandas.DataFrame(sample_type)
  sample_typedf = pandas.concat([sample_typedf,sample_typedf,sample_typedf])
  sample_typedf.reset_index(drop=True, inplace=True)

  # Y for expression value 
  y = {'y':[df[0].iloc[:,j][i]
      for j in range(9,12)
        for i in range(0, shape)]}

  #create df with Y 
  data_distrib = pandas.DataFrame(y)

  # X for ATG7/iso1/iso2
  data_distrib['x'] = ''
  data_distrib['x'][: shape] = 'ATG7'
  data_distrib['x'][shape: shape*2] = 'ATG7(1)'
  data_distrib['x'][shape*2:] = 'ATG7(2)'

  # Will be used for Hue
  data_distrib['Sample Type'] = sample_typedf['sample_type']

  #graph
  seaborn.catplot(x="x", y="y", hue="Sample Type", aspect= 1.2, kind="box", palette = 'Set2', data = data_distrib, 
                  boxprops={'lw':2}, medianprops={'lw':2}, whiskerprops={'lw':2}, showcaps=True, showfliers=False)
  
  plt.ylabel('Expression of ATG7 (TPM)')
  plt.xlabel('')
  plt.title('ATG7 expression in '+ df[1], x=.55)

In [ ]:
#Distribution: Stats for all 

all_dfs = [[df_ATG7_Normal, 'all normal tissue', df_ATG7_Primary, 'all primary tumors']]

isoforms = ['ATG7', 'ATG7_1', 'ATG7_2']

for df in all_dfs:
  for isoform in isoforms:

    n1 = "{:.3f}".format(numpy.median(df[0][isoform]))
    n2 = "{:.3f}".format(numpy.median(df[2][isoform]))

    # Mann-Whitney analysis 
    statistic, pvalue = scipy.stats.mannwhitneyu(df[0][isoform], df[2][isoform])
    Pvalue = "{:.3E}".format(pvalue)
    print("the pvalue between", df[1]+'('+n1+')', "and", df[3]+'('+n2+')', "for", isoform, 'is', Pvalue)
  print()


### 2) Normalized

In [ ]:
# NORMALiZED Distribution: Data preparation + graph

data = [[df_Norm_Prim, 'all_tissue']]


for df in data:
   
  shape = df[0].shape
  shape = shape[0]
  print(shape)

  #select the sample type column
  sample_type = {'sample_type':[df[0].iloc[:,1][i]
      for i in range(0, shape)]}

  #convert to df, and copy it 3 times
  sample_typedf = pandas.DataFrame(sample_type)
  sample_typedf = pandas.concat([sample_typedf,sample_typedf,sample_typedf])
  sample_typedf.reset_index(drop=True, inplace=True)

  # Y for expression value 
  y = {'y':[df[0].iloc[:,j][i]
      for j in range(18,20)
        for i in range(0, shape)]}

  #create df with Y 
  data_distrib = pandas.DataFrame(y)

  # X for iso1/ATG7 and iso2/ATG7
  data_distrib['x'] = ''
  data_distrib['x'][: shape] = 'ATG7(1)/ATG7'
  data_distrib['x'][shape:] = 'ATG7(2)/ATG7'


  # Will be used for Hue
  data_distrib['Sample Type'] = sample_typedf['sample_type']

  #graph
  seaborn.catplot(x="x", y="y", hue="Sample Type", aspect= 1.2, kind="box", palette = 'Set2', data = data_distrib, 
                  boxprops={'lw':2}, medianprops={'lw':2}, whiskerprops={'lw':2}, showcaps=True, showfliers=False)
  
  plt.ylabel('ATG7 isoforms/total ATG7 in TPM')
  plt.xlabel('')
  plt.title('ATG7 proportion in '+ df[1], x=0.55)


In [ ]:
#NORMALIZED Distribution: Stats for all 

all_dfs = [[df_ATG7_Normal, 'all normal tissue', df_ATG7_Primary, 'all primary tumors']]

isoforms = ['ATG7(1)norm', 'ATG7(2)norm']
print('NORMALIZED')

for df in all_dfs:
  for isoform in isoforms:

    n1 = "{:.3f}".format(numpy.median(df[0][isoform]))
    n2 = "{:.3f}".format(numpy.median(df[2][isoform]))

    # Mann-Whitney analysis 
    statistic, pvalue = scipy.stats.mannwhitneyu(df[0][isoform], df[2][isoform])
    Pvalue = "{:.3E}".format(pvalue)
    print("the pvalue between", df[1]+'('+n1+')', "and", df[3]+'('+n2+')', "for", isoform, 'is', Pvalue)
  print()


#IV] Expression correlation 

In [ ]:
#isoform correlation in all tissue 

#three values are very high and make my graph ugly... I remove them.
df_ATG7_Primary2 = Primary[Primary['ATG7(1)'] < 30 ]
print(Primary.shape)
print(Primary2.shape)


tissues = [[df_ATG7_Normal, "normal tissue"], [df_ATG7_Primary2, "primary tumors"]]

for tissue in tissues: 
  x = tissue[0]['ATG7(1)'].to_list()
  y = tissue[0]['ATG7(2)'].to_list()

  r_value, p_value = scipy.stats.spearmanr(x, y)
  print(r_value, p_value)

  seaborn.regplot(x=x,y=y, x_bins=800, x_ci = 0, marker="+")
  name = "Correlation between the two isoforms in "+  tissue[1]
  plt.title(name)
  plt.xlabel("ATG7(1) expression in tpm")
  plt.ylabel('ATG7(2) expression tpm')
  plt.show()
  plt.clf()


In [ ]:
#To see information. 
# → I can see many samples have no expression of ATG7(2) so I loose many data in normal tissue when removing <4tpm.
#If I want to work on Normal, I have to do exeption on iso2

all_tissue_used = [[df_normal_exp,'Normal Tissue'], [df_primary_exp,'Primary Tumors']]

isoforms = ['ATG7_1', 'ATG7_2']

for tissue in all_tissue_used:
  print(tissue[1])
  ff = tissue[0].shape
  print('number of patients:', ff[0])
  print("ATG7_1 max is", tissue[0]['ATG7_1'].max())
  print("ATG7_2 max is", tissue[0]['ATG7_2'].max())
    

  for isoform in isoforms:
    print('shape before removing', isoform, 'zero expression', tissue[0].shape)
    df_atleast1 = tissue[0][tissue[0][isoform]>=0]
    print('shape after removing', isoform, 'zero expression', df_atleast1.shape)

    #create a new row with max() for each gene
    tissue[0].loc['max()'] = tissue[0].max()

    #Transpose to have max() in column
    tissue[0] = tissue[0].T
    nb_before = tissue[0].shape
    tissue[0] = tissue[0][tissue[0]['max()'] > 1.2]
    nb_after = tissue[0].shape
    print("number of genes removed:", nb_before[0]-nb_after[0])
    print('now the minimum of TPM is', tissue[0]['max()'].min())

    #delete the column max()
    tissue[0] =  tissue[0].drop('max()', axis = 1) 
    # print(tissue[0].shape)

    #transpose back
    tissue[0] = tissue[0].T


    print()

In [ ]:
#to know the ensembl code for HK2
protein_coding_list.loc[protein_coding_list['hgnc_symbol'] == 'MFAP3']

In [ ]:
# Spearman Correlation, main genes 

all_tissue_used = [[df_normal_exp,'Normal Tissue'], [df_primary_exp,'Primary Tumors']]

HK2 = 'ENSG00000159399'
YAP1 = 'ENSG00000137693'
AJUBA = 'ENSG00000129474'
YBX1 = 'ENSG00000065978'
TGFB1 = 'ENSG00000105329'
ATG7_1 = 'ATG7_1'
ATG7_2 = 'ATG7_2'
MTDH = 'ENSG00000147649'
MAPK1 = 'ENSG00000100030'
KPNA1 = 'ENSG00000114030'
TNPO1 = 'ENSG00000083312'

for tissue in all_tissue_used:
  genes_x = [[KPNA1,'KPNA1'], [MAPK1, 'MAPK1'], [TNPO1,'TNPO1'], [MTDH,'MTDH'], 
             [HK2, "HK2"], [YAP1, 'YAP1'], [AJUBA,'AJUBA'], [YBX1,'YBX1'], 
             [TGFB1,'TGFB1'], [ATG7_1,'ATG7_1'], [ATG7_2,'ATG7_2']]

  print(tissue[1])

  for gene in genes_x:
    x = tissue[0][gene[0]].to_list()
    y = tissue[0]['ATG7'].to_list()
    y1 = tissue[0]['ATG7_1'].to_list()
    y2 = tissue[0]['ATG7_2'].to_list()  
  
    r_value, p_value = scipy.stats.spearmanr(x, y)
    print('  correlation between ATG7 and', gene[1],  'is:', 'r_value = {:.2f} for a p_value of {:.2e}'.format(r_value, p_value))
    r_value, p_value = scipy.stats.spearmanr(x, y1)
    print('  correlation between ATG7_1 and', gene[1],  'is:', 'r_value = {:.2f} for a p_value of {:.2e}'.format(r_value, p_value))
    r_value, p_value = scipy.stats.spearmanr(x, y2)
    print('  correlation between ATG7_2 and', gene[1],  'is:', 'r_value = {:.2f} for a p_value of {:.2e}'.format(r_value, p_value))
    print()

In [ ]:
# Spearman Correlation, all genes, PRIMARY TUMORS
all_tissue_used = [[df_primary_exp,'Primary Tumors']]

isoforms = ['ATG7_1', 'ATG7_2']

all_positiv_corr_spearman = []

for tissue in all_tissue_used:
  print(tissue[1], tissue[0].shape)

  #create a new row with max() for each gene and transpose
  tissue[0].loc['max()'] = tissue[0].max()
  tissue[0] = tissue[0].T

  #select all row where max() > 2.3 (= 5 tpm)
  nb_before = tissue[0]['max()'].count()
  tissue[0] = tissue[0][tissue[0]['max()'] > 2.3]
  nb_after = tissue[0]['max()'].count()
  print("number of genes removed:", nb_before-nb_after)
  print('the minimum of TPM is', tissue[0]['max()'].min())
  
  #delete the column max() and tranpose back
  tissue[0] =  tissue[0].drop('max()', axis = 1) 
  tissue[0] = tissue[0].T
  print('nombre genes after removing < 4tpm', tissue[0].shape)

  for isoform in isoforms:

    genepos = []
    geneneg = []
    rvaluepos = []
    rvalueneg = []
    pvaluepos = []
    pvalueneg = []

    print(isoform)
    x = tissue[0][isoform].to_list()
    genes = tissue[0].columns[3:]

    for gene in genes:
      y = tissue[0][gene].to_list()
      r_value, p_value = scipy.stats.spearmanr(x, y)

      if r_value > 0.65:
        genepos.append(gene)
        rvaluepos.append(r_value)
        pvaluepos.append(p_value)
        
      elif r_value < -0.65:
        geneneg.append(gene)
        rvalueneg.append(r_value)
        pvalueneg.append(p_value)

    #create a dictionnary
    d = dict(ensembl_gene_id= genepos, r_value=rvaluepos,  p_value=pvaluepos)
    d2 = dict(Gene= geneneg, r_value=rvalueneg,  p_value=pvalueneg)

    #Convert to dataframe
    positiv_corr = 'pos_corr_'+tissue[1]
    negativ_corr = 'neg_corr_'+tissue[1]

    df_positiv_corr = pandas.DataFrame.from_dict(d, orient='index')
    df_negativ_corr = pandas.DataFrame.from_dict(data=d2, orient='index')

    #Transpose to have a better table
    df_positiv_corr = df_positiv_corr.transpose()
    df_negativ_corr = df_negativ_corr.transpose()

    #name
    positiv_corr = 'pos_spearman_corr_'+tissue[1]+'_'+isoform
    negativ_corr = 'neg_spearman_corr_'+tissue[1]+'_'+isoform

    print(positiv_corr, df_positiv_corr.shape)
    print(negativ_corr, df_negativ_corr.shape)

    #scending orde, save to excel
    df_positiv_corr['info'] = tissue[1]+'_'+isoform
    df_positiv_corr = protein_coding_list.merge(df_positiv_corr)
    df_positiv_corr = df_positiv_corr.drop(['entrezgene_id', 'transcript_biotype'], axis = 1)

    df_positiv_corr = df_positiv_corr.sort_values(['r_value'], ascending = False)

    df_positiv_corr.to_excel(positiv_corr+'.xlsx', index=False)
    df_negativ_corr.to_excel(negativ_corr+'.xlsx', index=False)

    all_positiv_corr_spearman.append(df_positiv_corr)
  print()

In [ ]:
#Graphic representation, two graphs in one

all_tissue_used = [[df_primary_exp,'Primary Tumors']]

HK2 = 'ENSG00000159399'
YAP1 = 'ENSG00000137693'
AJUBA = 'ENSG00000129474'
YBX1 = 'ENSG00000065978'
TGFB1 = 'ENSG00000105329'
MAPK1 = 'ENSG00000100030'
MTDH = 'ENSG00000147649'
KPNA1 = 'ENSG00000114030'
TNPO1 = 'ENSG00000083312'


 
genes = [[ATG7,'ATG7'], [ATG7_1,'ATG7_1'], [MAPK1,'MAPK1'], [MTDH,'MTDH'], [KPNA1,'KPNA1'], 
         [TNPO1,'TNPO1'], [HK2,'HK2'], [YAP1,'YAP1'], [AJUBA,'AJUBA'], [YBX1,'YBX1'], [TGFB1,'TGFB1']]

for gene in genes:
  for df in data:
    
      fig, ax = plt.subplots(1,2, figsize=(18,8))
      name = gene[1]+ " correlation with the two isoforms"
      name2 = gene[1]+ " expression (log2 tpm)"

      fig.suptitle(name, fontsize="x-large")

      x = df[0][gene[0]].to_list()
      y1 = df[0]['ATG7_1'].to_list()
      y2 = df[0]['ATG7_2'].to_list()


      fig = seaborn.regplot(x=x,y=y1, ax=ax[0], x_bins=500, label = df[1], color = df[2])
      fig.set_title("ATG7(1)")
      fig.set_xlabel(name2)
      fig.set_ylabel('ATG7(1) expression (log2 tpm)')

      fig = seaborn.regplot(x=x,y=y2, ax=ax[1], x_bins=500, label = df[1], color = df[2])
      fig.set_title("ATG7(2)")
      fig.set_xlabel(name2)
      fig.set_ylabel('ATG7(2) expression (log2 tpm)')
      plt.legend()
      plt.show()
      plt.clf()

      print(df[1])
      r_value, p_value = scipy.stats.spearmanr(x, y1)
      print('  correlation between ATG7_1 and', gene[1],  'is:', 'r_value = {:.2f} for a p_value of {:.2e}'.format(r_value, p_value))
      r_value, p_value = scipy.stats.spearmanr(x, y2)
      print('  correlation between ATG7_2 and', gene[1],  'is:', 'r_value = {:.2f} for a p_value of {:.2e}'.format(r_value, p_value))
      print()